In [17]:
def isa_gen(loop=(0,0)):
    loop_inst = "00"
    loop_cnt = "00000"
    if(loop[0] == "start"):
        loop_inst = "10"
        loop_cnt = loop[1]
    elif (loop[0] == "end"):
        loop_inst = "01"
    elif (loop[0] == "both"):
        loop_inst = "11"
        loop_cnt = loop[1]
    else:
        loop_inst = "00"

    instruction = loop_inst + str(loop_cnt)
    
    return instruction

In [1]:
loop = (1,5)

In [14]:
isa_gen(loop=("start", 100))

'10100'

In [18]:
isa_gen()

'0000000'

In [29]:
import re

# -----------------------------------------------------------------------------
# 1) Parse an assembly line into {cmd: [operands], ...}
# -----------------------------------------------------------------------------
def parse_line(line):
    tokens = [tok.strip() for tok in line.split(',')]
    instr = {}
    for tok in tokens:
        m = re.match(r'(\w+)\s*(.*)', tok)
        if not m:
            continue
        cmd, operand = m.group(1), m.group(2).strip()
        if operand:
            instr.setdefault(cmd, []).append(operand)
        else:
            instr.setdefault(cmd, [])
    return instr

# -----------------------------------------------------------------------------
# 2) Encode parsed dict → 84-bit integer
# -----------------------------------------------------------------------------
def encode_instruction(instr):
    bits = ''

    # 1) Loop control
    bits += '1' if 'loop_start' in instr else '0'
    bits += '1' if 'loop_end'   in instr else '0'
    bits += f"{int(instr.get('loop_cnt', ['0'])[0]):08b}"

    bits += ' '
    print(bits)

    # 2) Input crossbar (8 bits)
    if 'input' in instr:
        s, e = map(int, re.findall(r'\d+', instr['input'][0]))
        bits += f"{s:04b}{e:04b}"
    else:
        bits += '0' * 8
    bits += ' '
    print(bits)
    
    # 3) Output crossbar (6 bits: 4 + 2)
    if 'output' in instr:
        s, e = map(int, re.findall(r'\d+', instr['output'][0]))
        bits += f"{s:04b}{e:02b}"
    else:
        bits += '0' * 6
    bits += ' '
    print(bits)
    
    # 4) Clock gating (2 bits)
    bits += '1' if 'cg1' in instr else '0'
    bits += '1' if 'cg2' in instr else '0'
    bits += ' '
    print(bits)
    
    # helper for DMEM mode bits
    mode_map = {'idle':'00', 'incr':'01', 'dec':'10', 'stay':'11'}

    # 5) DMEM1 port
    bits += instr.get('bank_sel1', ['0'])[0][-1]
    vw1 = instr.get('valid_w1', ['0'])[0][-1]
    bits += vw1
    if vw1 == '1':
        w1 = int(instr.get('waddr1', ['0'])[0])
        bits += f"{w1:09b}"
    else:
        mode1 = instr.get('wmode1', [None])[0]
        bits += '0' * 7 + mode_map.get(mode1, '00')
    vr1 = instr.get('valid_r1', ['0'])[0][-1]
    bits += vr1
    if vr1 == '1':
        r1 = int(instr.get('raddr1', ['0'])[0])
        bits += f"{r1:09b}"
    else:
        mode1r = instr.get('rmode1', [None])[0]
        bits += '0' * 7 + mode_map.get(mode1r, '00')
    bits += ' '
    print(bits)

    # 6) DMEM2 port
    bits += instr.get('bank_sel2', ['0'])[0][-1]
    vw2 = instr.get('valid_w2', ['0'])[0][-1]
    bits += vw2
    if vw2 == '1':
        w2 = int(instr.get('waddr2', ['0'])[0])
        bits += f"{w2:09b}"
    else:
        mode2 = instr.get('wmode2', [None])[0]
        bits += '0' * 7 + mode_map.get(mode2, '00')
    vr2 = instr.get('valid_r2', ['0'])[0][-1]
    bits += vr2
    if vr2 == '1':
        r2 = int(instr.get('raddr2', ['0'])[0])
        bits += f"{r2:09b}"
    else:
        mode2r = instr.get('rmode2', [None])[0]
        bits += '0' * 7 + mode_map.get(mode2r, '00')

    bits += ' '

    bits2 = bits.replace(" ","")

    # 6) Pad to 84 bits
    if len(bits2) < 84:
        bits += '0' * (84 - len(bits2))
    elif len(bits2) > 84:
        raise ValueError(f"Generated {len(bits)} bits, exceeds 84")

    return bits

# -----------------------------------------------------------------------------
# 3) Main: read 'test.txt' → write 'output.bin'
# -----------------------------------------------------------------------------
def assemble():
    with open('test.txt') as fin, open('output.bin','w') as fout:
        for line in fin:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            instr = parse_line(line)
            print(instr)
            val   = encode_instruction(instr)
            print(val)

if __name__ == '__main__':
    assemble()


{'loop_start': [], 'loop_cnt': ['10']}
1000001010 
1000001010 00000000 
1000001010 00000000 000000 
1000001010 00000000 000000 00 
1000001010 00000000 000000 00 000000000000000000000 
1000001010 00000000 000000 00 000000000000000000000 000000000000000000000 0000000000000000
{'input': ['cb0-3'], 'output': ['cb2-1']}
0000000000 
0000000000 00000011 
0000000000 00000011 001001 
0000000000 00000011 001001 00 
0000000000 00000011 001001 00 000000000000000000000 
0000000000 00000011 001001 00 000000000000000000000 000000000000000000000 0000000000000000
{'bank_sel1': ['1'], 'valid_w1': ['1'], 'waddr1': ['45']}
0000000000 
0000000000 00000000 
0000000000 00000000 000000 
0000000000 00000000 000000 00 
0000000000 00000000 000000 00 110001011010000000000 
0000000000 00000000 000000 00 110001011010000000000 000000000000000000000 0000000000000000
{'bank_sel1': ['0'], 'wmode1': ['incr']}
0000000000 
0000000000 00000000 
0000000000 00000000 000000 
0000000000 00000000 000000 00 
0000000000 00000000 